##Actualizar YahooFinance a última versión

In [110]:
pip install yfinance --upgrade

##Importar librerías

In [111]:
import pandas as pd, yfinance as yf, pytz
from datetime import datetime, timedelta
print('proceso terminado!'.upper())

PROCESO TERMINADO!


##Cláusulas para identificar tickers

In [112]:
# Opciones de pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('future.no_silent_downcasting', True)

# Crear una consulta para obtener las acciones que han cambiado su valor en los últimos 5 minutos
q = yf.EquityQuery('and', [
    yf.EquityQuery('eq', ['region', 'us']), # Acciones de la región de EE. UU. | eq: equal
    # yf.EquityQuery('is-in', ['exchange', 'NMS', 'NYQ']),
    yf.EquityQuery('gt', ['avgdailyvol3m', 950000]),  # Cambio porcentual mayor que 0
    yf.EquityQuery('btwn', ['intradayprice', 2.41, 25.9]), # btwn: between
    # yf.EquityQuery('lte', ['intradayprice', 25.9]), # lt: lower than
    # yf.EquityQuery('gte', ['intradayprice', 4.9]), # gt: greater than
    yf.EquityQuery('gt', ['percentchange', 2.81])
    # yf.EquityQuery('lte', ['percentchange', -8.99])
])

# Obtener los datos de las acciones que cumplen con la consulta
# response = yf.screen(q, size= 250,sortField='intradayprice', sortAsc=True)
response = yf.screen(q, size= 250,sortField='percentchange', sortAsc=False)

# Convertir la respuesta a un DataFrame
df = pd.DataFrame(response)
# print(df.shape)
# print(df.to_string())

# Convertir la columna quotes a un DataFrame
quotes_df = pd.DataFrame(df['quotes'].tolist())

# Concatenar el DataFrame original con el nuevo DataFrame de quotes
df = pd.concat([df, quotes_df], axis=1)

# Eliminar la columna quotes original
df = df.drop('quotes', axis=1)
# print(df.columns)

# Resetear el índice
#df = df.reset_index()

df = df[['regularMarketTime','symbol','displayName','longName','regularMarketPrice','regularMarketChangePercent','fiftyTwoWeekRange','fiftyDayAverage','averageDailyVolume3Month','quoteType','financialCurrency', 'marketCap','regularMarketVolume']]
df['prop_volumen'] = df['regularMarketVolume'] / df['averageDailyVolume3Month']

# df = df.query("(regularMarketPrice >= 2 & regularMarketPrice <= 30) & financialCurrency == 'USD' & quoteType == 'EQUITY' & marketCap >= 300000000 & prop_volumen >= 0.6")
# df = df.query("financialCurrency == 'USD' & quoteType == 'EQUITY' & marketCap >= 95000000 & & regularMarketVolume >= 999999 & prop_volumen >= 0.1").reset_index()
df = df.query("financialCurrency == 'USD' & quoteType == 'EQUITY'").reset_index()

# Convertir campos
df['regularMarketTime'] = pd.to_datetime(df['regularMarketTime'], unit='s', utc=True).dt.tz_convert('America/New_York') # Convertir a la zona horaria de Nueva York
df['marketCap'] = df['marketCap'].astype(int)

# print(df.shape)
# print(df.dtypes)

# Eliminar valores de la lista
stock_buscar = df['symbol'].tolist()
# stock_buscar = ['AMZN','MSTR','TSLA']
# valores_eliminar = ['AAIGF','BPAQF','ALTM','CSCCF','GLCNF','SSLZY','MUI','GSAT']
valores_eliminar = ['AAIGF','BPAQF','ALTM','CSCCF','GLCNF','SSLZY','MUI','GSAT','SBH','WKEY','OMI','DV','SITC','FLO','IDAI','KNWND','BHC','DENN','FNMA']
for valor in valores_eliminar:
    if valor in stock_buscar:
        stock_buscar.remove(valor)

# Validar
print(df.to_string())
print(f"Total: {len(stock_buscar)} | Nombre de instrumento: {stock_buscar}")
print('proceso terminado!'.upper())

     index         regularMarketTime symbol                    displayName                                   longName  regularMarketPrice  regularMarketChangePercent fiftyTwoWeekRange  fiftyDayAverage  averageDailyVolume3Month quoteType financialCurrency     marketCap  regularMarketVolume  prop_volumen
0        0 2025-02-28 16:00:01-05:00   TRNR           Interactive Strength                  Interactive Strength Inc.              2.5700                   67.973854    0.855 - 2100.0         2.270220                   6418155    EQUITY               USD       3855539            186310417     29.028657
1        1 2025-02-28 15:59:59-05:00    OMI                  Owens & Minor                        Owens & Minor, Inc.              9.6000                   39.332375      6.07 - 28.35        11.641800                   1068425    EQUITY               USD     740246400             11188581     10.472032
2        2 2025-02-28 16:00:02-05:00   ENVB            Enveric Biosciences              

##Descargar y formatear datos
*A las 10:35 se identifican mejor los ganadores del día x tendencia*

In [128]:
# Establecer la zona horaria de Nueva York
nueva_york = pytz.timezone('America/New_York')

# Establecer la fecha y hora de inicio y fin
start = datetime(2025, 2, 28, 0, 0, 0, tzinfo=nueva_york) # Año, Mes, Día, Hora, Minuto, Segundo, Zona horaria
# Restar 200 días a la fecha de hoy
# start = datetime.now(nueva_york) - timedelta(days=200)
# end = datetime(2025, 2, 21, 10, 20, 0, tzinfo=nueva_york) # A las 10:19 se identifican mejor las tendencias
end = datetime(2025, 2, 28, 13, 19, 0, tzinfo=nueva_york) # A las 9:49 primer reporte 15m | + 4 min para validar período 15m
# end = datetime.now()
#end = datetime.now() - timedelta(days=3) # Para ayer

# Crear un DataFrame vacío
df_final = pd.DataFrame(columns=['Ticker', 'Fecha', 'Close', 'Volume', 'High', 'Low', '%_cambio'])

# Crear un DataFrame vacío para almacenar los resultados de cambio_jornada
df_cambio_jornada = pd.DataFrame(columns=['Ticker', 'Cambio_Jornada'])

for a in stock_buscar:
    try:
        data = yf.download(tickers=a, start=start, end=end, interval="15m")
        print(f"\nData collection successful for: {a}")
    except:
        print("Failure in download")

    df1 = pd.DataFrame(data)

    # df1.index = df1.index.tz_localize(nueva_york)  # Comentado!
    df1.index = df1.index.tz_convert(nueva_york)  # Comentado!

    # Resetear el índice
    df1 = df1.reset_index()

    df1.columns = ['Fecha', 'Close', 'High', 'Low', 'Open', 'Volume']
    df1 = df1[['Fecha', 'Close', 'High', 'Low','Volume']]
    df1['Ticker'] = a

    # # Agregar columna para indicar si se cumple la condición de compra
    # df1['Compra'] = False

    # # Iterar sobre los grupos de tres filas
    # df1['Compra'] = False

    # # Iterar sobre los grupos de tres filas
    # for i in range(len(df1) - 2):
    #     if df1['Ticker'].iloc[i] == df1['Ticker'].iloc[i + 1] == df1['Ticker'].iloc[i + 2]:
    #         if df1['Close'].iloc[i+2] >= df1['Close'].iloc[i+1] and df1['Close'].iloc[i+2] > df1['Close'].iloc[i]:
    #             df1.loc[i + 2, 'Compra'] = True

    df1['%_cambio'] = df1['Close'].pct_change() * 100

    # Agregar resultado a df_final
    df_final = pd.concat([df_final.dropna(how='all'), df1.dropna(how='all')], ignore_index=True)

    # Agregar resultado a df_cambio_jornada
    cambio_jornada = round(df1.groupby('Ticker')['%_cambio'].sum(), 2)
    df_cambio_jornada = pd.concat([df_cambio_jornada.dropna(how='all'), pd.DataFrame({'Ticker': [a], 'Cambio_Jornada': [cambio_jornada.iloc[0]]})], ignore_index=True)

# Mensaje
print('proceso terminado!'.upper())

[*********************100%***********************]  1 of 1 completed
<ipython-input-128-4ebaa68f78ae>:53: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_final = pd.concat([df_final.dropna(how='all'), df1.dropna(how='all')], ignore_index=True)
<ipython-input-128-4ebaa68f78ae>:57: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_cambio_jornada = pd.concat([df_cambio_jornada.dropna(how='all'), pd.DataFrame({'Ticker': [a], 'Cambio_Jornada': [cambio_jornada.iloc[0]]})], ignore_index=True)
[********


Data collection successful for: TRNR

Data collection successful for: ENVB


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed



Data collection successful for: PEPG

Data collection successful for: HRTX


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


...


Data collection successful for: CRSR

Data collection successful for: SEAT

Data collection successful for: AMC


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed



Data collection successful for: AVDL

Data collection successful for: SGMT

Data collection successful for: NNOX


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Data collection successful for: INTC

Data collection successful for: DK

Data collection successful for: PPBT
PROCESO TERMINADO!


##Calcular VWAP (Volume-Weighted Average Price)

In [129]:
# Fuente VWAP Definition: https://tinyurl.com/22dp95x6

# Valor típico
df_final['TypicalPrice'] = (((df_final['High'] + df_final['Low'] + df_final['Close']) / 3) * df_final['Volume'])

# Convertir a númerico
df_final['TypicalPrice'] = pd.to_numeric(df_final['TypicalPrice'], errors='coerce')
df_final['Volume'] = pd.to_numeric(df_final['Volume'], errors='coerce')

# Cálculo de VWAP
df_final['VWAP'] = df_final.groupby('Ticker')['TypicalPrice'].cumsum() / df_final.groupby('Ticker')['Volume'].cumsum()
# df_final['vChk'] = df_final['Close'] > df_final['VWAP']

# Filtrar por condición de VWAP = True
# ultimas_filas = df_final.sort_values('Fecha').groupby('Ticker').tail(1)
# tickers_a_eliminar = ultimas_filas[~ultimas_filas['vChk']]['Ticker'].unique()
# df_final = df_final[~df_final['Ticker'].isin(tickers_a_eliminar)]

# print(df_final.to_string())
# df_final.query("Ticker == 'MPW'")

# Mensaje
print('proceso terminado!'.upper())

PROCESO TERMINADO!


##Calcular EMA 9, 20 y 200 (Exponential Moving Average)
*Los filtros / True sería: Pr close > EMA 200 & VWAP & EMA 20 (& EMA 9[Opt])*

In [130]:
# Fuente What is Exponential Moving Average? Definition, Formula, Calculation, and Trading: https://www.strike.money/technical-analysis/ema
# Fuente Calculating EMA for Tesla Stock Using Pandas: https://tinyurl.com/2bjlzbjp
# Fuente Simple EMA Crossover trading signals using pandas: https://tinyurl.com/2ayxfqjl

# Función para calcular las EMA
def calcular_ema(df, ticker, periodo):
    return df.groupby(ticker)['Close'].transform(lambda x: x.ewm(span=periodo, adjust=False).mean())

# EMA's a calcular según período
df_final['EMA9'] = calcular_ema(df_final, 'Ticker', 9)
df_final['EMA20'] = calcular_ema(df_final, 'Ticker', 20)
df_final['EMA200'] = calcular_ema(df_final, 'Ticker', 200)

# Columna True
df_final['Compra'] = (df_final['Close'] > df_final['VWAP']) & (df_final['Close'] > df_final['EMA200']) & (df_final['Close'] >= df_final['EMA20']) & (df_final['Close'] >= df_final['EMA9'])

# Mensaje
print('proceso terminado!'.upper())

PROCESO TERMINADO!


##Df final con datos [Yf DB]

In [131]:
# Mostrar los datos descargados
# print(df_cambio_jornada.to_string())
df_final.head(15)

# Consulta
# stock = ['tuya','rily', 'agh']  # Use a list instead of a tuple
# df_final[df_final['Ticker'].str.upper().isin([s.upper() for s in stock])]

# Mensaje
# print('proceso terminado!'.upper())

,Ticker,Fecha,Close,Volume,High,Low,%_cambio,TypicalPrice,VWAP,EMA9,EMA20,EMA200,Compra
0,TRNR,2025-02-28 09:30:00-05:00,1.8495,55568936,2.2799,1.7700,NaN,1.092745e+08,1.966467,1.849500,1.849500,1.849500,False
1,TRNR,2025-02-28 09:45:00-05:00,1.8094,5162776,1.9900,1.8000,-2.168152,9.636149e+06,1.957966,1.841480,1.845681,1.849101,False
2,TRNR,2025-02-28 10:00:00-05:00,1.6499,3434744,1.8300,1.5700,-8.815077,5.781705e+06,1.943263,1.803164,1.827035,1.847119,False
3,TRNR,2025-02-28 10:15:00-05:00,1.7100,1125459,1.7400,1.6403,3.642650,1.909641e+06,1.939014,1.784531,1.815889,1.845754,False
4,TRNR,2025-02-28 10:30:00-05:00,1.7900,1352482,1.8100,1.6800,4.678358,2.380368e+06,1.935381,1.785625,1.813423,1.845200,False
5,TRNR,2025-02-28 10:45:00-05:00,1.8150,1135844,1.8300,1.7400,1.396653,2.038840e+06,1.933029,1.791500,1.813573,1.844899,False
6,TRNR,2025-02-28 11:00:00-05:00,1.8400,1008875,1.8700,1.7301,1.377409,1.829460e+06,1.931274,1.801200,1.816090,1.844850,False
7,TRNR,2025-02-28 11:15:00-05:00,2.0200,1802754,2.0500,1.8200,9.782606,3.539407e+06,1.932093,1.844960,1.835510,1.846593,True
8,TRNR,2025-02-28 11:30:00-05:00,2.2568,12588325,2.3700,1.9800,11.722770,2.772285e+07,1.972980,1.927328,1.875633,1.850675,True
9,TRNR,2025-02-28 11:45:00-05:00,1.9499,13429587,2.4000,1.8700,-13.598897,2.784356e+07,1.986926,1.931842,1.882706,1.851662,False


##Top 10 de mayores cambios

In [132]:
df_cambio_jornada = df_cambio_jornada.sort_values('Cambio_Jornada',ascending=False).reset_index(drop=True)
df_cambio_jornada.head(15)

,Ticker,Cambio_Jornada
0,TRNR,44.88
1,RILY,15.83
2,PEPG,15.58
3,HRTX,12.46
4,BTDR,12.28
5,NUKK,11.51
6,LTBR,9.71
7,SOUN,9.70
8,ACHR,9.34
9,WULF,8.07


##Df Númerico [Transpuesto]

In [107]:
# Agrupar por 'Ticker' y 'Fecha', y luego pivotar
pivot_df = df_final.pivot_table(index='Ticker', columns='Fecha', values='%_cambio', aggfunc='first', fill_value=False)

# Mostrar el DataFrame resultante
# print(pivot_df.to_string())
# print(df_cambio_jornada.to_string())

# Unir DataFrames a partir de la columna Ticker
df_unido = pd.merge(pivot_df, df_cambio_jornada, on='Ticker')
df_unido['symbol'] = df_unido['Ticker']
df_unido = pd.merge(df_unido, df[['symbol', 'regularMarketPrice']], on='symbol')

# Take profit
df_unido['TP'] = ((df_unido['regularMarketPrice'] * 0.0166) + df_unido['regularMarketPrice']).round(2).astype(str)
# Stop Loss
df_unido['SL'] = ((df_unido['regularMarketPrice'] * 0.0125) - df_unido['regularMarketPrice']).round(2).astype(str)

# Precio
df_unido['regularMarketPrice'] = df_unido['regularMarketPrice'].astype(str)

# Renombrar columnas
df_unido = df_unido.rename(columns={'regularMarketPrice': 'Precio'})
df_unido = df_unido.rename(columns={'Cambio_Jornada': 'Cambio'})
df_unido = df_unido.drop('symbol', axis=1)

# Ordenar
df_unido = df_unido.sort_values('Cambio',ascending=False).reset_index(drop=True)

# Define una función que extrae la hora y el minuto de un nombre de columna
def extraer_hora_minuto(columna):
    return columna.split(' ')[1][:5]

# Selecciona solo las columnas que cumplan con la condición
columnas_fecha = df_unido.filter(regex=r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}-\d{2}:\d{2}').columns

# Crea nuevas columnas con la hora y el minuto
for columna in columnas_fecha:
    # Convierte el nombre de la columna a texto
    nombre_columna = str(columna)
    # Extrae la hora y el minuto
    hora_minuto = extraer_hora_minuto(nombre_columna)
    # Renombra la columna con la hora y el minuto
    df_unido = df_unido.rename(columns={columna: hora_minuto})

# Definir una función para colorear las celdas con valor True
def colorear_celdas(val):
    if isinstance(val, (int, float)):
        if val > 1.5:
            color = '#385a3c'  # verde más fuerte
        elif val > 0.33:
            color = '#70b578'  # verde muy claro
        elif val > -0.33:
            color = '#FFFFFF'  # fondo blanco
        elif val > -1.5:
            color = '#ff6666'  # rojo muy claro
        else:
            color = '#990000'  # rojo muy oscuro
    else:
        color = 'black'
    return f'background-color: {color}'

# Consultas
# print(df_unido.to_string())
df_unido = df_unido.head(15)
# stock = ['fold', 'coty', 'sono']  # Use a list instead of a tuple
# df_unido = df_unido[df_unido['Ticker'].str.upper().isin([s.upper() for s in stock])]
# df_unido = df_unido.query("T >= '2'")

# Aplicar estilo a dataframe
styled_df = df_unido.style.map(colorear_celdas)
styled_df # Mostrar el resultado

# Seleccionar las 14 últimas columnas, conservando la primera
# columnas_seleccionadas = ['Ticker'] + list(df_unido.columns[-12:])
# styled_df_seleccionado = df_unido[columnas_seleccionadas].style.map(colorear_celdas) # Mostrar columnas seleccionadas
# styled_df_seleccionado # Mostrar el resultado

,Ticker,09:45,10:00,10:15,10:30,10:45,11:00,11:15,11:30,11:45,12:00,12:15,12:30,12:45,13:00,13:15,13:30,13:45,14:00,14:15,14:30,14:45,15:00,15:15,15:30,15:45,Cambio,Precio,TP,SL
0,TRNR,-2.168152,-8.815077,3.642650,4.678358,1.396653,1.377409,9.782606,11.722770,-13.598897,15.390531,16.013336,-6.715708,10.061607,2.115671,-9.748974,6.077169,3.053441,-2.962969,-5.744268,5.486939,1.727450,-4.550943,3.190477,-0.034483,-1.502379,39.880000,2.57,2.61,-2.54
1,PEPG,0.763102,3.765687,7.661293,2.434450,0.914080,-1.086955,0.717946,0.014550,0.363636,4.166667,-7.130432,2.808982,-0.182145,0.364963,-0.727272,-0.366300,0.000000,0.551466,1.133460,-0.578454,0.363636,0.724637,6.482018,-0.344572,7.118645,29.930000,3.15,3.2,-3.11
2,NEXT,0.417830,1.248268,0.547945,4.972749,0.259571,-0.453072,-0.390120,-0.391641,0.393181,0.391648,-0.065021,0.455431,-0.259067,0.714290,1.934237,1.328273,-1.435708,0.848637,-0.276310,1.133497,1.120799,0.061578,0.246159,-0.306944,1.231532,13.730000,8.22,8.36,-8.12
3,SOUN,3.560211,5.460060,4.410355,-2.846652,-0.368616,-0.300732,0.434854,3.446708,-1.683341,0.558923,-1.018987,-0.514743,-1.314204,-0.122020,0.525893,1.874173,-1.785646,-0.164158,0.979934,-0.028240,0.035781,0.000000,-0.371767,0.939971,1.544220,13.250000,10.85,11.03,-10.71
4,BTDR,0.969077,5.438760,3.511051,-0.963147,0.634248,0.290758,-0.708859,1.308015,-0.048315,0.748382,1.166348,0.408832,-1.384365,0.908345,0.572829,1.301870,-0.722893,-0.728149,0.651996,0.809712,0.401608,-0.160004,-1.522433,-0.838077,0.845160,12.890000,12.31,12.51,-12.16
5,LTBR,8.419963,5.944390,1.402712,1.249445,0.327015,-2.829879,-3.255884,4.019668,0.808627,-0.534763,-1.433690,-2.227272,-2.310552,0.128494,1.235733,1.314557,-1.946247,0.661623,0.469485,0.559816,0.837371,-0.460831,-0.277775,-0.928509,1.405807,12.580000,10.85,11.03,-10.71
6,ACHR,4.949243,1.390565,2.019083,0.067794,1.867977,0.286693,-2.744423,3.092293,1.562501,-1.572144,0.970898,-0.961562,-1.825451,0.232428,-0.753618,-0.116825,-1.111114,-0.591368,1.665679,2.867172,-0.625715,-0.057242,0.101955,0.665986,0.885518,12.270000,8.875,9.02,-8.76
7,NUKK,-1.785714,4.727270,2.847228,2.093177,3.835978,-4.076429,0.929611,-0.032895,-0.374465,2.159455,5.528610,-0.153184,-2.239949,-1.946016,-0.448783,-0.899686,-2.206361,-1.194422,1.242441,0.995029,1.510670,0.724688,-1.104901,0.746994,0.709218,11.590000,15.7,15.96,-15.5
8,WULF,2.732094,4.699198,1.972883,-0.483687,-0.364517,-1.226832,-0.609914,4.099378,-1.431980,-0.242136,0.000000,-0.364074,-1.705242,0.991325,0.613499,0.365850,-1.458079,0.000000,2.096179,1.932365,0.118486,0.118346,-0.709225,-0.357140,0.119477,10.910000,4.19,4.26,-4.14
9,NVAX,2.465023,-0.065021,4.489262,0.498132,1.982650,-1.093550,-0.009830,-0.297332,3.142331,-0.716840,0.180498,-1.381379,-0.426308,-0.795114,0.308268,0.892433,-0.275345,0.183255,0.487804,-0.362859,1.277687,-0.240523,0.361660,-0.659462,0.845220,10.790000,8.33,8.47,-8.23


##Df categórico [Transpuesto]

In [108]:
# Agrupar por 'Ticker' y 'Fecha', y luego pivotar
pivot_df = df_final.pivot_table(index='Ticker', columns='Fecha', values='Compra', aggfunc='first', fill_value=False)

# Mostrar el DataFrame resultante
pivot_df

# Unir DataFrames a partir de la columna Ticker
df_unido = pd.merge(pivot_df, df_cambio_jornada, on='Ticker')
df_unido['symbol'] = df_unido['Ticker']
df_unido = pd.merge(df_unido, df[['symbol', 'regularMarketPrice']], on='symbol')

# Take profit
df_unido['TP'] = ((df_unido['regularMarketPrice'] * 0.0166) + df_unido['regularMarketPrice']).round(2).astype(str)
# Stop Loss
df_unido['SL'] = ((df_unido['regularMarketPrice'] * 0.0125) - df_unido['regularMarketPrice']).round(2).astype(str)

# Precio
df_unido['regularMarketPrice'] = df_unido['regularMarketPrice'].astype(str)

# Renombrar columnas
df_unido = df_unido.rename(columns={'regularMarketPrice': 'Precio'})
df_unido = df_unido.rename(columns={'Cambio_Jornada': 'Cambio'})
df_unido = df_unido.drop('symbol', axis=1)

# Seleccionar columnas que contienen el caracter ":" y contar True
columnas_con_punto_y_coma = [col for col in df_unido.columns if ":" in str(col)]
df_unido['T'] = df_unido[columnas_con_punto_y_coma].apply(lambda x: x.sum(), axis=1).astype(str)

# Ordenar
df_unido = df_unido.sort_values('Cambio',ascending=False).reset_index(drop=True)

# Define una función que extrae la hora y el minuto de un nombre de columna
def extraer_hora_minuto(columna):
    return columna.split(' ')[1][:5]

# Selecciona solo las columnas que cumplan con la condición
columnas_fecha = df_unido.filter(regex=r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}-\d{2}:\d{2}').columns

# Crea nuevas columnas con la hora y el minuto
for columna in columnas_fecha:
    # Convierte el nombre de la columna a texto
    nombre_columna = str(columna)
    # Extrae la hora y el minuto
    hora_minuto = extraer_hora_minuto(nombre_columna)
    # Renombra la columna con la hora y el minuto
    df_unido = df_unido.rename(columns={columna: hora_minuto})

# Definir una función para colorear las celdas con valor True
def colorear_celdas(val):
    if isinstance(val, bool):
        color = '#599060' if val else 'white'
    elif isinstance(val, (int, float)):
        if val > 3:
            color = '#50989c'
        elif val > 1:
            color = '#70b578'  # verde más intenso
        elif val > 0:
            color = '#525252'  # verde claro
        else:
            color = '#ff6666'
    else:
        color = 'black'
    return f'background-color: {color}'

# Consultas
df_unido = df_unido.head(15)
# stock = ['fold', 'coty', 'sono']  # Use a list instead of a tuple
# df_unido = df_unido[df_unido['Ticker'].str.upper().isin([s.upper() for s in stock])]
# df_unido = df_unido.query("T >= '2'")

# Aplicar estilo a dataframe
styled_df = df_unido.style.map(colorear_celdas)
styled_df # Mostrar el resultado

# Seleccionar las 14 últimas columnas, conservando la primera
# columnas_seleccionadas = ['Ticker'] + list(df_unido.columns[-16:])
# styled_df_seleccionado = df_unido[columnas_seleccionadas].style.map(colorear_celdas) # Mostrar columnas seleccionadas
# styled_df_seleccionado # Mostrar el resultado

,Ticker,09:30,09:45,10:00,10:15,10:30,10:45,11:00,11:15,11:30,11:45,12:00,12:15,12:30,12:45,13:00,13:15,13:30,13:45,14:00,14:15,14:30,14:45,15:00,15:15,15:30,15:45,Cambio,Precio,TP,SL,T
0,TRNR,False,False,False,False,False,False,False,True,True,False,True,True,True,True,True,True,True,True,True,False,True,True,False,True,True,True,39.880000,2.57,2.61,-2.54,16
1,PEPG,False,True,True,True,True,True,True,True,True,True,True,False,True,True,True,True,False,False,True,True,True,True,True,True,True,True,29.930000,3.15,3.2,-3.11,22
2,NEXT,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,13.730000,8.22,8.36,-8.12,25
3,SOUN,False,True,True,True,True,True,True,True,True,True,True,True,True,False,False,False,True,False,False,True,True,True,True,False,True,True,13.250000,10.85,11.03,-10.71,19
4,BTDR,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,False,False,12.890000,12.31,12.51,-12.16,22
5,LTBR,False,True,True,True,True,True,True,False,True,True,True,True,False,False,False,False,True,False,False,False,True,True,True,True,False,True,12.580000,10.85,11.03,-10.71,16
6,ACHR,False,True,True,True,True,True,True,True,True,True,True,True,True,False,False,False,False,False,False,False,True,True,True,True,True,True,12.270000,8.875,9.02,-8.76,18
7,NUKK,False,False,True,True,True,True,True,True,True,True,True,True,True,True,True,False,False,False,False,False,False,True,True,True,True,True,11.590000,15.7,15.96,-15.5,18
8,WULF,False,True,True,True,True,True,True,True,True,True,True,True,True,False,True,True,True,False,False,True,True,True,True,True,True,True,10.910000,4.19,4.26,-4.14,22
9,NVAX,False,True,True,True,True,True,True,True,True,True,True,True,True,True,False,False,True,True,True,True,True,True,True,True,True,True,10.790000,8.33,8.47,-8.23,23


In [109]:
# import pytz
# from datetime import datetime
# import time

# # Convertir la fecha y hora a una zona horaria "desconocida"
# from datetime import datetime, timedelta

# # Get the current date and time
# now = datetime.now()

# # Subtract 5 hours
# five_hours_ago = now - timedelta(hours=5)

# # Format the result as a string
# result = five_hours_ago.strftime("%Y-%m-%d_%H-%M-%S")

# print(result)

# # Exportar el archivo con la fecha y hora sin zona horaria
# pivot_df.to_csv(f'archivo_{result}.csv')